<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/BigQuery_datawarehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [141]:
from google.colab import auth
auth.authenticate_user()
print("authorized")

authorized


# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and displays the result, optionally saving it to a variable as a `DataFrame`.

In [0]:
%%bigquery --project e6820-235222 df
create table warehouse.tweet(
tweets STRING
)

# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [0]:
project_id = 'e6820-235222'

### Sample approximately 2000 random rows

In [4]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `warehouse.tweet`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `warehouse.tweet`''').to_dataframe()

print(df,"\n", row_count)

        tweets
0       string
1  hello world
2  hello world
3  hello world
4       string
5       string 
 6


### Describe the sampled data

In [5]:
df.describe()

,tweets
count,6
unique,2
top,hello world
freq,3



### View the first 10 rows

In [6]:
df.head(10)

,tweets
0,string
1,hello world
2,hello world
3,hello world
4,string
5,string


In [7]:
# 10 highest total_precipitation samples
df.sort_values('tweets', ascending=False).head(10)

,tweets
0,string
4,string
5,string
1,hello world
2,hello world
3,hello world


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [8]:
import pandas as pd

df = pd.io.gbq.read_gbq('''
  SELECT *
  FROM `warehouse.tweet`
  GROUP BY tweets
  LIMIT 10
''', project_id=project_id, verbose=False, dialect='standard')

df.head()

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:810: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


,tweets
0,hello world
1,string


In [63]:
!pip install kafka-python

In [0]:
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
from json import dumps, loads
import time

In [0]:
consumer = KafkaConsumer(
    'tweepyv1',
     bootstrap_servers=['35.243.144.79:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group31333',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [0]:
%%bigquery --project e6820-235222 df
create table warehouse.tweet_info(
favorite_count int64,
friends_count int64,
followers_count int64,
full_text STRING
)

In [152]:
message = consumer.poll(timeout_ms=10000, max_records=1)
for key in message.keys():
  if type(key) is not str:
    try:
      message[str(key)] = message[key]
    except:
      try:
        message[repr(key)] =messagemydict[key]
      except:
        pass
    del message[key]
    
loads(dumps(message))["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]
# print(type(loads(dumps(message))["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]))

{'contributors': None,
 'coordinates': {'coordinates': [-97.1397, 49.8873], 'type': 'Point'},
 'created_at': 'Sun Apr 28 19:48:51 +0000 2019',
 'entities': {'hashtags': [{'indices': [71, 80], 'text': 'cocobolo'},
   {'indices': [81, 91], 'text': 'fretboard'},
   {'indices': [92, 100], 'text': 'luthier'}],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/1…',
    'expanded_url': 'https://twitter.com/i/web/status/1122588505987788800',
    'indices': [102, 125],
    'url': 'https://t.co/arHCs4R4nA'}],
  'user_mentions': []},
 'extended_tweet': {'display_text_range': [0, 175],
  'entities': {'hashtags': [{'indices': [71, 80], 'text': 'cocobolo'},
    {'indices': [81, 91], 'text': 'fretboard'},
    {'indices': [92, 100], 'text': 'luthier'},
    {'indices': [101, 116], 'text': 'electricguitar'},
    {'indices': [117, 130], 'text': 'customguitar'}],
   'symbols': [],
   'urls': [{'display_url': 'instagram.com/p/Bwz70JYgtPU/…',
     'expanded_url': 'https://www.instagram.c

In [153]:
# client.insert_rows("`warehouse.tweet`", str(string), [('tweets','STRING')])

res = loads(dumps(message))
query_params = [bigquery.ScalarQueryParameter('tweets', 'STRING', 'message{}'.format(1))]
queryjob = "INSERT warehouse.tweet_info (favorite_count, friends_count, followers_count, full_text) VALUES ({}, {}, {}, '".format(res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['favorite_count'], \
res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['user']['friends_count'], \
res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['user']['followers_count']) + \
str(res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['text']) \
+ "')"

job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params
client.query(
    queryjob,
    job_config=job_config
)


In [115]:
res = loads(dumps(message))
print(res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['favorite_count'])
print("{}, {}, {}".format(res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['favorite_count'], \
res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['user']['friends_count'], \
res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['user']['followers_count']) )

0
0, 12640, 13496


In [154]:
i = 1
for line in consumer:
  res = loads(dumps(line))
  query_params = [bigquery.ScalarQueryParameter('tweets', 'STRING', 'message{}'.format(i))]
  query_job = "INSERT warehouse.tweet_info (favorite_count, friends_count, followers_count, full_text) VALUES ({}, {}, {}, '".format(res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['favorite_count'], \
  res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['user']['friends_count'], \
  res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['user']['followers_count']) + \
  str(res["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]['text']) \
  + "')"

  job_config = bigquery.QueryJobConfig()
  job_config.query_parameters = query_params
  client.query(
      query_job,
      job_config=job_config
  )

  i += 1

TypeError: ignored

In [120]:
print(client.query('''
  SELECT *
  FROM `warehouse.tweet_info` limit 100
  ''').to_dataframe())

   favorite_count  friends_count  followers_count  \
0               0          12640            13496   

                                   full_text  
0  @minhtngo @jadorkables Ignorance abounds.  


In [24]:
type("string")

str